In [1]:
import pickle
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import cluster, metrics

file = open("../log/pingpong (1).pickle", "rb")
data = pickle.load(file)
file.close()
data['command'].pop()
scene_info = data['scene_info']
scene_command = data['command']
scene_command = scene_command + [('NONE', 'NONE')]

for i in range(2, 75):
    path = "../log/pingpong (" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    del data['command'][0]
    data['command'].pop()
    scene_info = scene_info + data['scene_info']
    scene_command = scene_command + [('NONE', 'NONE')]
    scene_command = scene_command + data['command']
    scene_command = scene_command + [('NONE', 'NONE')]
    file.close()
    
print(len(scene_info))
print(len(scene_command))
print(scene_command)

62395
62395
[['SERVE_TO_RIGHT', 'SERVE_TO_LEFT'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['MOVE_LEFT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['NONE', 'NONE'], ['MOVE_LEFT', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['NONE', 'NONE'], [None, None], ['MOVE_LEFT', 'NONE'], ['MOVE_LEFT', 'NONE'], ['NONE', 'NONE'], ['MOVE_RIGHT', 'NONE'], ['MOVE_RI

In [2]:
print(scene_command[1154])

['MOVE_RIGHT', 'NONE']


In [3]:
Ball_x = []
Ball_y = []
Ball_speed_x = []
Ball_speed_y = []
Platform_1P = []
Platform_2P = []
blocker_x = []
blocker_y = []
command = []
count = 0

for i, s in enumerate(scene_info[1:-2]):
    Ball_x.append(s['ball'][0])
    Ball_y.append(s['ball'][1])
    Ball_speed_x.append(s['ball_speed'][0])
    Ball_speed_y.append(s['ball_speed'][1])
    Platform_1P.append(s['platform_1P'][0])
    Platform_2P.append(s['platform_2P'][0])
    blocker_x.append(s['blocker'][0])
    blocker_y.append(s['blocker'][1])
    
for i, s in enumerate(scene_command[1:-2]):
    if s[0] == "NONE" and s[1] == "NONE":
        command.append(0)
    elif s[0] == "NONE" and s[1] == "MOVE_LEFT":
        command.append(1)
    elif s[0] == "NONE" and s[1] == "MOVE_RIGHT":
        command.append(2)
    elif s[0] == "MOVE_LEFT" and s[1] == "NONE":
        command.append(3)
    elif s[0] == "MOVE_LEFT" and s[1] == "MOVE_LEFT":
        command.append(4)
    elif s[0] == "MOVE_LEFT" and s[1] == "MOVE_RIGHT":
        command.append(5)
    elif s[0] == "MOVE_RIGHT" and s[1] == "NONE":
        command.append(6)
    elif s[0] == "MOVE_RIGHT" and s[1] == "MOVE_LEFT":
        command.append(7)
    elif s[0] == "MOVE_RIGHT" and s[1] == "MOVE_RIGHT":
        command.append(8)
    else:
        command.append(0)

numpy_feature_data = np.array([Ball_x, Ball_y, Ball_speed_x, Ball_speed_y, Platform_1P, Platform_2P, blocker_x, blocker_y])
feature = np.transpose(numpy_feature_data) 
numpy_answer_data = np.array([command])
answer = np.transpose(numpy_answer_data).ravel()

print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98  80  -7 ...  80   0 240]
 [ 91  87  -7 ...  80   5 240]
 [ 84  94  -7 ...  80  10 240]
 ...
 [195 109 -18 ...  95  75 240]
 [177  91 -18 ... 100  80 240]
 [159  73 -18 ... 105  85 240]]
(62392, 8)
[6 6 6 ... 2 2 2]
(62392,)


In [8]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3, 4, 5, 6, 7, 8, 9]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=9, test_size=0.3, random_state=9)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 9 folds for each of 9 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:   34.0s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   35.4s finished


In [10]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 9}
[[2336  344  330  283    7   16  284   14    2]
 [ 672 1153  193   30   27   14   39  112    2]
 [ 671  205 1051   27    9   93   25    8   32]
 [ 613   35   37 1079   48  111  184   10    5]
 [ 157  153   24  148  196  152   31  108    3]
 [ 181   32  187  188   78 1573   40   15   56]
 [ 649   32   40  179    7   10 1014  100   34]
 [ 167  227   21   37   51   14  197 1566   47]
 [ 163   35  149   33   10  114  135  121  143]]
              precision    recall  f1-score   support

           0       0.42      0.65      0.51      3616
           1       0.52      0.51      0.52      2242
           2       0.52      0.50      0.51      2121
           3       0.54      0.51      0.52      2122
           4       0.45      0.20      0.28       972
           5       0.75      0.67      0.71      2350
           6       0.52      0.49      0.51      2065
           7       0.76      0.67      0.71      2327
           8       0.44      0.16      0.23       903

    ac